## Übung 3b
#### CNN für Cats und Dog Bilder

In [19]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
from pathlib import Path
tf.random.set_seed(42)

In [45]:
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [2]:
data_dir = pathlib.Path("../data/cats_vs_dogs/")

In [4]:
[e.name for e in data_dir.glob("./*")]

['valid', 'test', 'train']

In [5]:
!mv ../data/cats_vs_dogs/validation ../data/cats_vs_dogs/valid
!mv ../data/cats_vs_dogs/evaluation ../data/cats_vs_dogs/test
!rm -rf ../data/cats_vs_dogs/data

mv: rename ../data/cats_vs_dogs/validation to ../data/cats_vs_dogs/valid/validation: No such file or directory
mv: rename ../data/cats_vs_dogs/evaluation to ../data/cats_vs_dogs/test/evaluation: No such file or directory


In [6]:
[e.name for e in data_dir.glob("./*")]

['valid', 'test', 'train']

In [7]:
CLASS_NAMES = [item.name for item in data_dir.glob('train/*')]
CLASS_NAMES

['cat', 'dog']

In [9]:
n_labels = 5000

train_cats = list(data_dir.glob('train/cat/*.jpg'))[:n_labels//2]
train_dogs = list(data_dir.glob('train/dog/*.jpg'))[:n_labels//2]
train_files = train_cats + train_dogs
train_files = [str(x) for x in train_files]

In [10]:
train_ds = tf.data.Dataset.list_files(train_files)
valid_ds = tf.data.Dataset.list_files(str(data_dir/'valid/*/*.jpg'))
test_ds  = tf.data.Dataset.list_files(str(data_dir/'test/*/*.jpg'))

In [11]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    label = 1 if parts[-2] == "dog" else 0
    return label

In [12]:
def decode_img(img, IMG_WIDTH=160, IMG_HEIGHT=160):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32)
    img = (img/127.5) - 1.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

In [13]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [14]:
train_ds = train_ds.map(process_path)
valid_ds = valid_ds.map(process_path)
test_ds  = test_ds.map(process_path)

In [15]:
BATCH_SIZE = 32
 
train_ds = train_ds.batch(BATCH_SIZE)
valid_ds = valid_ds.batch(BATCH_SIZE)
test_ds  = test_ds.batch(BATCH_SIZE)

Das erste CNN besteht bei mir aus einem zweidimensionalen Convoluional Layer mit Pixelgröße 3x3. Darauf folgt ein Max Pooliny Layer mit Größe 4x4. Danach habe ich einen Leaky ReLu Layer eingebaut, gefolgt von einem Dropout Layer mit Rate 0,25. Danach flatte ich den Input und dann folgt noch ein Dense Layer mit der Sigmoid Aktivierungsfunktion. Vor dem Output Layer habe ich noch einen Dropout Layer mit Rate 0,5 eingebaut. Als Ouptut Layer habe ich einen Denselayer mit der Softmax Aktivierung gewählt, der mir die Wahrscheinlichkeite für Katze und Hund ausgibt.

In [53]:
input_shape = [160, 160, 3]

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
#model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(4, 4)))
model.add(keras.layers.LeakyReLU())
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation='sigmoid'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(2, activation='softmax'))

    # compile model
opt      = keras.optimizers.SGD(lr=0.01, momentum=0.9)
loss_fct = keras.losses.sparse_categorical_crossentropy
metrics  = [keras.metrics.sparse_categorical_accuracy]
model.compile(optimizer=opt, loss=loss_fct, metrics=metrics)

In [54]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 158, 158, 32)      896       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 39, 39, 32)        0         
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 39, 39, 32)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 39, 39, 32)        0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 48672)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               6230144   
_________________________________________________________________
dropout_16 (Dropout)         (None, 128)             

In [55]:
cnn_train_hist_1 = model.fit(train_ds, validation_data=valid_ds, epochs=10)

Train for 157 steps, validate for 157 steps
Epoch 1/10
157/157 [==============================] - 86s 547ms/step - loss: 0.7053 - sparse_categorical_accuracy: 0.5574 - val_loss: 0.6382 - val_sparse_categorical_accuracy: 0.6252
Epoch 2/10
157/157 [==============================] - 84s 536ms/step - loss: 0.6394 - sparse_categorical_accuracy: 0.6266 - val_loss: 0.6156 - val_sparse_categorical_accuracy: 0.6522
Epoch 3/10
157/157 [==============================] - 84s 536ms/step - loss: 0.6109 - sparse_categorical_accuracy: 0.6576 - val_loss: 0.5974 - val_sparse_categorical_accuracy: 0.6698
Epoch 4/10
157/157 [==============================] - 84s 534ms/step - loss: 0.5684 - sparse_categorical_accuracy: 0.6980 - val_loss: 0.5645 - val_sparse_categorical_accuracy: 0.7174
Epoch 5/10
157/157 [==============================] - 84s 536ms/step - loss: 0.5405 - sparse_categorical_accuracy: 0.7280 - val_loss: 0.5451 - val_sparse_categorical_accuracy: 0.7258
Epoch 6/10
157/157 [=====================

In [61]:
loss1, accuracy1 = model.evaluate(test_ds)

7/7 [==============================] - 1s 180ms/step - loss: 0.5676 - sparse_categorical_accuracy: 0.7350


In [62]:
print("Test loss: {:.2f}".format(loss1))
print("Test accuracy: {:.2f}".format(accuracy1))

initial loss: 0.57
initial accuracy: 0.74


Durch dieses CNN kommt ich auf eine Accuracy von 74%.
Nun versuche ich noch ein weiteres CNN aufzubauen.
Dies habe ich etwas abgeändert. Zu Beginn habe ich 2 Concoluional Layer, mit unterschiedlichen Größen und einmal der Tanh und einmal der ReLu Aktivierunsgfunktion. Gefolgt von einem Maxpooling, einem Dropout und einem Flatten Layer und dann kommt bereits der Output Layer, der gleichbleibt.

In [47]:
model2 = keras.models.Sequential()
model2.add(keras.layers.Conv2D(64, (5, 5), activation='tanh', input_shape=input_shape))
model2.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
model2.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(keras.layers.Dropout(0.25))
model2.add(keras.layers.Flatten())
#model2.add(keras.layers.Dense(16, activation='sigmoid'))
#model2.add(keras.layers.Dropout(0.5))
model2.add(keras.layers.Dense(2, activation='softmax'))

    # compile model
opt2      = keras.optimizers.Adam()
loss_fct2 = keras.losses.categorical_crossentropy
metrics2  = [keras.metrics.sparse_categorical_accuracy]
model2.compile(optimizer=opt2, loss=loss_fct2, metrics=metrics2)

In [48]:
model2.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 156, 156, 64)      4864      
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 155, 155, 32)      8224      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 77, 77, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 77, 77, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 189728)            0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 379458    
Total params: 392,546
Trainable params: 392,546
Non-trainable params: 0
________________________________________________

In [49]:
cnn_train_hist_2 = model2.fit(train_ds, validation_data=valid_ds, epochs=10)

Train for 157 steps, validate for 157 steps
Epoch 1/10
157/157 [==============================] - 385s 2s/step - loss: 5.2948 - sparse_categorical_accuracy: 0.5020 - val_loss: 13.5974 - val_sparse_categorical_accuracy: 0.4982
Epoch 2/10
157/157 [==============================] - 369s 2s/step - loss: 65.8578 - sparse_categorical_accuracy: 0.5002 - val_loss: 115.6653 - val_sparse_categorical_accuracy: 0.4580
Epoch 3/10
157/157 [==============================] - 376s 2s/step - loss: 267.2621 - sparse_categorical_accuracy: 0.4908 - val_loss: 379.1777 - val_sparse_categorical_accuracy: 0.5034
Epoch 4/10
157/157 [==============================] - 383s 2s/step - loss: 420.5625 - sparse_categorical_accuracy: 0.5004 - val_loss: 530.1227 - val_sparse_categorical_accuracy: 0.4978
Epoch 5/10
157/157 [==============================] - 417s 3s/step - loss: 728.6656 - sparse_categorical_accuracy: 0.4996 - val_loss: 1440.3544 - val_sparse_categorical_accuracy: 0.5026
Epoch 6/10
157/157 [==============

In [66]:
loss2, accuracy2 = model2.evaluate(test_ds)

7/7 [==============================] - 3s 360ms/step - loss: 1235.0332 - sparse_categorical_accuracy: 0.5300


In [67]:
print("Test loss: {:.2f}".format(loss2))
print("Test accuracy: {:.2f}".format(accuracy2))

initial loss: 1235.03
initial accuracy: 0.53


Mit diesem CNN erreiche deutlich schwächere Ergebnisse und komme nur auf eine TestAccuracy von 53 %.
Nun erstelle ich noch ein drittes CNN, wieder mit etwas mehr Layern.

In [58]:
model3 = keras.models.Sequential()
model3.add(keras.layers.Conv2D(32, (3, 3), activation='tanh', input_shape=input_shape))
#model3.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model3.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model3.add(keras.layers.LeakyReLU(alpha = 0.3))
model3.add(keras.layers.Flatten())
model3.add(keras.layers.Dropout(0.50))
model3.add(keras.layers.Dense(128, activation='sigmoid'))
model3.add(keras.layers.Dropout(0.5))
model3.add(keras.layers.Dense(2, activation='softmax'))

    # compile model
opt3      = keras.optimizers.SGD(lr=0.01, momentum=0.9)
loss_fct3 = keras.losses.sparse_categorical_crossentropy
metrics3  = [keras.metrics.sparse_categorical_accuracy]
model3.compile(optimizer=opt3, loss=loss_fct3, metrics=metrics3)

In [59]:
model3.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 158, 158, 32)      896       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 79, 79, 32)        0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 79, 79, 32)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 199712)            0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 199712)            0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               25563264  
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)             

In [60]:
cnn_train_hist_3 = model3.fit(train_ds, validation_data=valid_ds, epochs=10)

Train for 157 steps, validate for 157 steps
Epoch 1/10
157/157 [==============================] - 119s 759ms/step - loss: 0.6998 - sparse_categorical_accuracy: 0.5700 - val_loss: 0.6474 - val_sparse_categorical_accuracy: 0.6176
Epoch 2/10
157/157 [==============================] - 119s 757ms/step - loss: 0.6440 - sparse_categorical_accuracy: 0.6270 - val_loss: 0.6535 - val_sparse_categorical_accuracy: 0.6054
Epoch 3/10
157/157 [==============================] - 119s 760ms/step - loss: 0.6021 - sparse_categorical_accuracy: 0.6672 - val_loss: 0.5922 - val_sparse_categorical_accuracy: 0.6806
Epoch 4/10
157/157 [==============================] - 120s 765ms/step - loss: 0.5736 - sparse_categorical_accuracy: 0.7002 - val_loss: 0.5979 - val_sparse_categorical_accuracy: 0.6762
Epoch 5/10
157/157 [==============================] - 119s 758ms/step - loss: 0.5417 - sparse_categorical_accuracy: 0.7288 - val_loss: 0.5846 - val_sparse_categorical_accuracy: 0.6872
Epoch 6/10
157/157 [================

In [68]:
loss3, accuracy3 = model3.evaluate(test_ds)

7/7 [==============================] - 1s 167ms/step - loss: 0.5864 - sparse_categorical_accuracy: 0.7400


In [69]:
print("Test loss: {:.2f}".format(loss3))
print("Test accuracy: {:.2f}".format(accuracy3))

initial loss: 0.59
initial accuracy: 0.74


Damit konnte ich mein erstes CNN noch leicht verbessern auf 74%.